In [1]:
import pandas as pd
import numpy as np
import plotly.express as px


In [2]:
# trades_filename = "/media/corvin/BigData/Crypto/Trades/EOSUSDT-trades-2020-12.zip"
# data = pd.read_csv(trades_filename,header = None, 
#                    names = ["trade Id", "price", "qty", "quoteQty", "time", "isBuyerMaker", "isBestMatch"] )
# data

In [3]:
def load_trades_from_path(path) :
    result = pd.read_csv(path,header = None, 
                   names = ["trade Id", "price", "qty", "quoteQty", "time", "isBuyerMaker"] )
    result["time"] = pd.to_datetime(result["time"], unit = "ms")
    return result

def get_path_daily_trades_by_contract(symbol, day, month, year, contract) :
    if contract not in ["coin", "usdt", "spot"] :
        print("Contract needs to be in ", ["coin", "usdt", "spot"])
        return
    contract_dict = {"coin" : "USD_PERP", "usdt" : "USDT", "spot" : None}
    contract_prefix = contract_dict[contract]
    day = int(day)
    if day <= 9 :
        day = "0" +str(day)
    else :
        day = str(day)
    if month <= 9 :
        month = "0" +str(month)
    else :
        month = str(month)
    
    root_path = "/media/corvin/BigData/Crypto/Trades/daily/"
    return root_path + symbol + contract_prefix +"-trades-" + str(year) +"-" + str(month) + "-" + str(day) + ".zip"

def load_daily_trades_coin_margined(symbol, day, month,year) :
    return load_trades_from_path(get_path_daily_trades_by_contract(symbol, day,month, year,"coin"))
def load_daily_trades_usdt_margined(symbol, day, month,year) :
    return load_trades_from_path(get_path_daily_trades_by_contract(symbol, day,month, year, "usdt"))

In [4]:
usdt_margined = load_daily_trades_usdt_margined("EOS", 1, 6, 2021)
coin_margined = load_daily_trades_coin_margined("EOS", 1, 6, 2021)

In [ ]:
# test = usdt_margined.resample("1000ms", on = "time").agg({"price" : np.mean})

In [5]:
def resample_prices(df) :
    return df.resample("1000ms", on = "time").agg({"price" : np.mean})

In [6]:
def split_into_buy_sell(df) :
    buys = (df[~df["isBuyerMaker"]])[["time", "price"]]
    sells = (df[df["isBuyerMaker"]])[["time", "price"]]
    return buys,sells

In [ ]:
usdt_margined_buys, usdt_margined_sells = split_into_buy_sell(usdt_margined)
coin_margined_buys, coin_margined_sells = split_into_buy_sell(coin_margined)

In [ ]:
usdt_margined_buys = resample_prices(usdt_margined_buys)
usdt_margined_sells = resample_prices(usdt_margined_sells)
coin_margined_buys = resample_prices(coin_margined_buys)
coin_margined_sells = resample_prices(coin_margined_sells)


In [ ]:
delta_coin_buy_usdt_sell = usdt_margined_sells - coin_margined_buys

In [ ]:
delta_coin_buy_usdt_sell.rename(columns = {"price" : "price_delta"},inplace = True)
delta_coin_buy_usdt_sell["coin_buy"] = coin_margined_buys["price"]

In [ ]:
delta_coin_buy_usdt_sell.head()


In [ ]:
def to_perc(row) :
    return row["price_delta"] / row["coin_buy"] * 100
    
delta_coin_buy_usdt_sell["perc"] = delta_coin_buy_usdt_sell.apply(to_perc,axis = 1)

In [ ]:
delta_coin_buy_usdt_sell.head()

In [ ]:
delta_coin_buy_usdt_sell[delta_coin_buy_usdt_sell["perc"] > 0.1]

In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure()
# fig.add_trace(go.Scatter(x=usdt_margined_sells["time"], y=usdt_margined_sells["price"],
#                     mode='lines+markers',
#                     name='USDT Sells'))
# fig.add_trace(go.Scatter(x=usdt_margined_buys["time"], y=usdt_margined_buys["price"],
#                     mode='lines+markers',
#                     name='USDT Buys'))
# fig.add_trace(go.Scatter(x=random_x, y=random_y1,
#                     mode='lines+markers',
#                     name='lines+markers')) 

In [ ]:
# fig = px.line(usdt_margined_buys.reset_index(), x="time", y=["price"],
# #               hover_data={"time": "|%B %d, %Y"},
# #               color = "isBuyerMaker"
#              )
# # fig.update_xaxes(
# #     dtick="MIN1",
# #     tickformat="%b\n%d")
# fig.show()

In [ ]:
usdt_margined.head()

In [ ]:
zoomed_in = usdt_margined[(usdt_margined["time"] >= "2021-06-01 09:54:50	")
                          & (usdt_margined["time"] <= "2021-06-01 09:55:30	") ].reset_index()
zoomed_in

In [ ]:
fig = px.line(zoomed_in, x="time", y=["price"],
#               hover_data={"time": "|%B %d, %Y"},
              color = "isBuyerMaker"
             ).update_traces(mode='lines+markers')
# fig.update_xaxes(
#     dtick="MIN1",
#     tickformat="%b\n%d")
fig.show()

In [20]:
    def zoom_in(data, start, end) :
        return data[(data["time"] >= start)
                              & (data["time"] <= end) ].reset_index()



In [9]:
coin_margined.head()

,trade Id,price,qty,quoteQty,time,isBuyerMaker
0,14584877,6.633,200.0,301.522690,2021-06-01 00:00:03.536,True
1,14584878,6.631,33.0,49.766249,2021-06-01 00:00:03.581,True
2,14584879,6.630,5.0,7.541478,2021-06-01 00:00:04.247,True
3,14584880,6.632,415.0,625.753920,2021-06-01 00:00:04.447,False
4,14584881,6.631,19.0,28.653295,2021-06-01 00:00:08.126,False


In [10]:
usdt_margined.head()

,trade Id,price,qty,quoteQty,time,isBuyerMaker
0,219254536,6.628,8.1,53.686,2021-06-01 00:00:01.500,False
1,219254537,6.628,2.0,13.256,2021-06-01 00:00:01.500,False
2,219254538,6.628,10.0,66.280,2021-06-01 00:00:01.500,False
3,219254539,6.627,28.0,185.556,2021-06-01 00:00:01.510,True
4,219254540,6.626,54.9,363.767,2021-06-01 00:00:01.510,True


In [11]:
start_time = "2021-06-01 09:54:50"
end_time = "2021-06-01 09:55:20	"


In [21]:
import plotly.graph_objects as go
zoomed_usdtm_buys, zoomed_usdtm_sells = split_into_buy_sell(zoom_in(usdt_margined, start_time, end_time))
zoomed_coinm_buys, zoomed_coinm_sells = split_into_buy_sell(zoom_in(coin_margined, start_time, end_time))



In [23]:
zoomed_coinm_sells.tail()

,time,price
65,2021-06-01 09:55:17.670,6.181
66,2021-06-01 09:55:17.670,6.181
67,2021-06-01 09:55:17.670,6.181
68,2021-06-01 09:55:17.675,6.181
69,2021-06-01 09:55:17.675,6.181


In [25]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=zoomed_usdtm_buys["time"], y=zoomed_usdtm_buys["price"],
                    mode='lines+markers',
                    name='USDTM Buys'
                        ))
fig.add_trace(go.Scatter(x=zoomed_usdtm_sells["time"], y=zoomed_usdtm_sells["price"],
                    mode='lines+markers',
                    name='USDTM Sells'
                        ))
fig.add_trace(go.Scatter(x=zoomed_coinm_buys["time"], y=zoomed_coinm_buys["price"],
                    mode='lines+markers',
                    name='CoinM Buys'
                        ))
fig.add_trace(go.Scatter(x=zoomed_coinm_sells["time"], y=zoomed_coinm_sells["price"],
                    mode='lines+markers',
                    name='CoinM Sells'
                        ))



# fig.add_trace(go.Scatter(x=zoomed_coinm["time"], y=usdt_margined_buys["price"],
#                     mode='lines+markers',
#                     name='USDT Buys'))
# fig.add_trace(go.Scatter(x=random_x, y=random_y1,
#                     mode='lines+markers',
#                     name='lines+markers')) 